# HW09: Transformers

Remember that these homework work as a completion grade. **You can skip one section of this homework.**

In [1]:
import warnings; warnings.simplefilter('ignore')
import pandas as pd
import nltk
df = pd.read_csv('train.csv')

df.columns = ["label", "title", "lead"]
label_map = {1:"world", 2:"sport", 3:"business", 4:"sci/tech"}

def replace_label(x):
	return label_map[x]
df["label_name"] = df["label"].apply(replace_label)
df['label']=df['label']-1
df["text"] = df["title"] + " " + df["lead"]
df = df.sample(n=10000) # # only use 10K datapoints
df.head()

,label,title,lead,label_name,text
28991,0,New Orleans Urges People to Take Shelter,NEW ORLEANS - With 135-mph Hurricane Ivan clos...,world,New Orleans Urges People to Take Shelter NEW O...
27936,1,SI.com,ST. LOUIS (Ticker) -- It looked like Roger Cle...,sport,SI.com ST. LOUIS (Ticker) -- It looked like Ro...
6849,3,Underwater Archaeologists Dig for History (AP),"AP - Instead of wearing khakis, students this ...",sci/tech,Underwater Archaeologists Dig for History (AP)...
29448,1,"Football? No, it #39;s the Ryder Cup","It #39;s got to be the Bobby Bowden, good-ol #...",sport,"Football? No, it #39;s the Ryder Cup It #39;s ..."
19419,1,Bellevue dominates dream matchup against De La...,"At almost noon yesterday, the Bellevue High Sc...",sport,Bellevue dominates dream matchup against De La...


## Hugginface Transformers

In [2]:
from transformers import TFDistilBertForSequenceClassification, DistilBertConfig,DistilBertTokenizerFast
import tensorflow as tf

config = DistilBertConfig.from_pretrained('distilbert-base-uncased')
config.num_labels = 4
transformer_model = TFDistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', config=config)
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')


Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForSequenceClassification: ['activation_13', 'vocab_transform', 'vocab_projector', 'vocab_layer_norm']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier', 'dropout_19', 'pre_classifier']
You should probably TRAIN this model on a down-stream task to be able to use i

In [3]:
##TODO build a transformer model to do sequence classification with the goal to predict the label from the text

max_len = 30

input_ids = tf.keras.layers.Input(shape=(max_len,), name='input_token', dtype='int32')
input_masks_ids = tf.keras.layers.Input(shape=(max_len,), name='masked_token', dtype='int32')
X = transformer_model(input_ids, input_masks_ids)
model = tf.keras.Model(inputs=[input_ids, input_masks_ids], outputs = X)

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


In [4]:
##TODO print the summary of the model
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_token (InputLayer)        [(None, 30)]         0                                            
__________________________________________________________________________________________________
masked_token (InputLayer)       [(None, 30)]         0                                            
__________________________________________________________________________________________________
tf_distil_bert_for_sequence_cla TFSequenceClassifier 66956548    input_token[0][0]                
Total params: 66,956,548
Trainable params: 66,956,548
Non-trainable params: 0
__________________________________________________________________________________________________


In [5]:
##TODO compile the model

model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam', 
              metrics=['accuracy']) 

In [6]:
##TODO split the sample into a training and a test set 
##TODO prepare the dataset for tensorflow.

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df['text'].tolist(), df['label'].tolist(), test_size=.2)
X_train_tf = [tokenizer(x, return_tensors="tf", padding=True, truncation=True, max_length=max_len) for x in X_train]

In [7]:
len(X_train_tf[562]["attention_mask"][0].numpy())
lengths = [len(x["input_ids"][0].numpy()) for x in X_train_tf]
min(lengths)

19

In [8]:
##TODO fit the model and print the obtained accuracy

input_ids, input_masks = [x["input_ids"][0].numpy() for x in X_train_tf if len(x["input_ids"][0].numpy())==max_len], [x["attention_mask"][0].numpy() for x in X_train_tf if len(x["attention_mask"][0].numpy())==max_len]
y_train = [y for y, l in zip(y_train, lengths) if l==max_len]
dataset = tf.data.Dataset.from_tensor_slices(({'input_token': input_ids, 'masked_token': input_masks}, y_train)).batch(8)

model_info = model.fit(dataset, epochs=1)

969/969 [==============================] - 1180s 1s/step - loss: 1.4345 - accuracy: 0.2397


In [9]:
model_info.history

{'loss': [1.4344974756240845], 'accuracy': [0.23970569670200348]}

## Generating Text

In [10]:
#!pip install transformers

from transformers import GPT2LMHeadModel, GPT2Tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2", pad_token_id=tokenizer.eos_token_id)

##TODO Pick one snippet for each label and generate some starting from the first 4-5 words
snippets = df[["label", "text"]].groupby("label").first().to_dict(orient="records")    
snippets

[{'text': "New Orleans Urges People to Take Shelter NEW ORLEANS - With 135-mph Hurricane Ivan closing in with frightening intensity, this flood-prone city scrambled Wednesday to get people out of harm's way, putting the frail and elderly in the cavernous Louisiana Superdome and urging others to move to higher floors in tall buildings.    Ivan made a slight turn north on a path that could bring it ashore early Thursday at the Alabama-Mississippi line, near Mobile, Ala., but forecasters said everyone from New Orleans to the Panhandle should be worried..."},
 {'text': 'SI.com ST. LOUIS (Ticker) -- It looked like Roger Clemens , Lance Berkman and Jeff Bagwell were leading the Houston Astros to an easy victory, but it turned out to be not so easy.'},
 {'text': 'Stock option expensing required next year The independent body charged with setting US accounting standards on Thursday gave hard deadlines for companies to treat stock options as expenses in financial statements.'},
 {'text': 'Under

In [11]:


for snip in snippets:
    print('  ')
    print('--------------------------')
    print('Original Text: ' + snip['text'])
    print('  ')
    
    list_txt = snip['text'].split(' ')
    input = ' '.join(list_txt[:5])
    input_ids = tokenizer.encode(input, return_tensors='pt')
    print('Input: ' + input)
    print('  ')
    print('-----------')
    print('GREEDY')
    print('-----------')
    greedy_output = model.generate(input_ids, max_length=50)
    print(tokenizer.decode(greedy_output[0], skip_special_tokens=True))
    
    print('  ')
    print('-----------')
    print('BEAM')
    print('-----------')
    beam_output = model.generate(
    input_ids, 
    max_length=50, 
    num_beams=5, 
    no_repeat_ngram_size=2, # to avoid repetitions of the same word sequences
    early_stopping=True
    )
    print(tokenizer.decode(beam_output[0], skip_special_tokens=True))
    
    print('  ')
    print('-----------')
    print('SAMPLING')
    print('-----------')
    sample_output = model.generate(
    input_ids, 
    do_sample=True, 
    max_length=50, 
    top_p=0.92, 
    top_k=0
    )
    print(tokenizer.decode(sample_output[0], skip_special_tokens=True))

  
--------------------------
Original Text: New Orleans Urges People to Take Shelter NEW ORLEANS - With 135-mph Hurricane Ivan closing in with frightening intensity, this flood-prone city scrambled Wednesday to get people out of harm's way, putting the frail and elderly in the cavernous Louisiana Superdome and urging others to move to higher floors in tall buildings.    Ivan made a slight turn north on a path that could bring it ashore early Thursday at the Alabama-Mississippi line, near Mobile, Ala., but forecasters said everyone from New Orleans to the Panhandle should be worried...
  
Input: New Orleans Urges People to
  
-----------
GREEDY
-----------
New Orleans Urges People to Stop Using the 'F' word'

The city of New Orleans is urging people to stop using the 'F' word, which is used to describe people who are not in the U.S.


  
-----------
BEAM
-----------
New Orleans Urges People to Stand Up to Police Brutality

The city of New Orleans is urging people to stand up to police 

In [12]:
## pick the generating model that looks best to you (and explain why) and set the length of each generated document to 50